In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import copy
import numpy as np                                              
import pandas as pd                                             
import matplotlib.pyplot as plt                                 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder               
import torchvision.transforms as transforms              
from torch.utils.data import SubsetRandomSampler 
import torch.utils.data as data   
from torchsummary import summary
import torch.nn.functional as F
from time import time

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from torchvision import models
from torch import nn
vgg = models.vgg16_bn(pretrained=True)
for param in vgg.features.parameters():
    param.requires_grad = False

In [ ]:
# Unfreeze upto 34
# ct = 0
# for child in vgg.children():
#     ct += 1
#     if ct < 34:
#         for param in child.parameters():
#             param.requires_grad = False

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#Convolutional Layer with Batch Norm
vgg.features[43] = nn.Sequential(vgg.features[43],nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
#                 
#Two Convolutional Layer with Batch Norm
# vgg.features[43] = nn.Sequential(vgg.features[43],nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
                
print(nn.Sequential(*list(vgg.children())[:-1]))

In [ ]:
#Loading Data 
my_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.ToPILImage(),
  transforms.Resize((224,224)),
  transforms.ToTensor(),
  transforms.Normalize(mean = [0.66445047,0.55465436,0.447036], std = [0.321551,0.33547384,0.3524585])
  ])
dataset =  torchvision.datasets.ImageFolder(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Clean Data\Clean Data", transform=my_transform)

train_data, valid_data, test_data = torch.utils.data.random_split(dataset, [9600,2400,3000], generator=torch.Generator().manual_seed(0))

train_loader = data.DataLoader(train_data, batch_size=64,shuffle= True)                  
valid_loader = data.DataLoader(valid_data, batch_size=64,shuffle= False)
test_loader = data.DataLoader(test_data, batch_size=64,shuffle= False)

In [ ]:
# Relabel Functions
def relabel_state(argument):
    switcher = {
        0: 0,
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 1,
        6: 1,
        7: 1,
        8: 1,
        9: 1,
        10:2,
        11:2,
        12:2,
        13:2,
        14:2}
    return switcher[argument.item()]

def relabel_type(argument):
    switcher = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 0,
        6: 1,
        7: 2,
        8: 3,
        9: 4,
        10:0,
        11:1,
        12:2,
        13:3,
        14:4}
    return switcher[argument.item()]

In [ ]:
#Transfer Learning Multi Head
class MyVgg(nn.Module):
    def __init__(self,originalmodel):
        super(MyVgg,self).__init__()
        vgg = originalmodel
        # Here you get the bottleneck/feature extractor
        self.vgg_feature_extractor = nn.Sequential(*list(vgg.children())[:-1])
        self.classifier1 = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 3),
            nn.Sigmoid()
        )
        self.classifier2 = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Linear(4096, 5),
            nn.Sigmoid()
                            )

    # Set your own forward pass
    def forward(self, img, extra_info=None):
        
        x = self.vgg_feature_extractor(img)
        x = x.view(x.size(0), -1)
        x1 = self.classifier1(x)
        x2 = self.classifier2(x)

        return x1, x2

In [ ]:
# Extra MLP Layer for State
# class MyVggextra(nn.Module):
#     def __init__(self,originalmodel):
#         super(MyVggextra,self).__init__()
#         vgg = originalmodel
#         # Here you get the bottleneck/feature extractor
#         self.vgg_feature_extractor = nn.Sequential(*list(vgg.children())[:-1])
#         self.classifier1 = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 3),
#             nn.Sigmoid()
#         )
#         self.classifier2 = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 5),
#             nn.Sigmoid()
#                             )
# 
#     # Set your own forward pass
#     def forward(self, img, extra_info=None):
#         
#         x = self.vgg_feature_extractor(img)
#         x = x.view(x.size(0), -1)
#         x1 = self.classifier1(x)
#         x2 = self.classifier2(x)
# 
#         return x1, x2

In [ ]:
# Convolutional Layer on each head
# class MyVggeach(nn.Module):
#     def __init__(self,originalmodel):
#         super(MyVggeach,self).__init__()
#         vgg = originalmodel
#         # Here you get the bottleneck/feature extractor
#         self.vgg_feature_extractor = nn.Sequential(*list(vgg.children())[:-1])
#         self.convlayer1 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
#         self.convlayer2 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), nn.BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
#                 nn.ReLU(inplace=True),nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False))
#         self.classifier1 = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 3),
#             nn.Sigmoid()
#         )
#         self.classifier2 = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 4096),
#             nn.ReLU(True),
#             nn.Linear(4096, 5),
#             nn.Sigmoid()
#                             )
# 
#     # Set your own forward pass
#     def forward(self, img, extra_info=None):
#         
#         x = self.vgg_feature_extractor(img)
#         x1= self.convlayer1(x)
#         x2= self.convlayer2(x)
#         x1 = x.view(x1.size(0), -1)
#         x2 = x.view(x2.size(0), -1)
#         #torch.Size([64, 25088])
#         x1 = self.classifier1(x1)
#         x2 = self.classifier2(x2)
# 
#         return x1, x2

In [ ]:
#MODEL ARGUMENTS
lr = 0.01
epochs = 10
eval_every = 60
model = MyVgg(vgg)
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=lr)
loss_fnc= torch.nn.CrossEntropyLoss()
loss_fnc =loss_fnc.to(device)


#OUTPUT DATA ARGUMENTS
seed = 1
torch.manual_seed(seed)
save= True
confusion=True
plot = True
statistics = True

In [ ]:
train_loss_list = []
train_acc_list = []
train_loss_list_state = []
train_acc_list_state = []
train_loss_list_type = []
train_acc_list_type = []

valid_loss_list = []
valid_acc_list = []
valid_loss_list_state = []
valid_acc_list_state = []
valid_loss_list_type = []
valid_acc_list_type = []

epoch_num = []         #Epoch number
start = time()
for e in range(epochs):
    print("Time:    ", time() - start)
    running_loss = []
    running_accuracy = []
    running_loss_state = []
    running_accuracy_state = []
    running_loss_type = []
    running_accuracy_type = []

    running_valid_loss = []
    running_valid_accuracy = []
    running_valid_loss_state = []
    running_valid_accuracy_state = []
    running_valid_loss_type = []
    running_valid_accuracy_type = []
    model.train()
  #               LOOKING AT TRAINING DATA

    for i, data in enumerate(train_loader):

            #get batch of data
        inputs, label = data[0].to(device), data[1].to(device)
    
        label_state = torch.zeros(len(label)).to(device)
        label_type = torch.zeros(len(label)).to(device)
        for k in range(len(label)):
          label_state[k] = relabel_state(label[k])
          label_type[k] = relabel_type(label[k])
        
        # Setting gradients to zero and running model on batch
        optimizer.zero_grad()
        predictions_state, predictions_type = model(inputs)

        # Computing loss based on Cross Entropy
        batch_loss_state = loss_fnc(input=predictions_state.squeeze(), target=label_state.long())
        batch_loss_type = loss_fnc(input=predictions_type.squeeze(), target=label_type.long())

        # Combining losses and calculating gradients
        Overall_loss = batch_loss_state + batch_loss_type
        Overall_loss.backward()
        optimizer.step()
        # Evaluating Training Acc
        _, predicted_state = torch.max(predictions_state.data, 1)
        _,predicted_type = torch.max(predictions_type.data, 1)
        trainAcc_state = (label_state == predicted_state).sum().item() / 64
        trainAcc_type = (label_type == predicted_type).sum().item() / 64

        if i % eval_every == 0:
           print("epoch: {} {} |||   loss type:   {}  trainAcc type:   {} |||  loss state {}  trainAcc state:  {}  ".format(e+1,i, batch_loss_type, trainAcc_type, batch_loss_state, trainAcc_state))

        running_loss.append(Overall_loss.item())
        running_accuracy.append((trainAcc_state+trainAcc_type)/2)
        running_loss_state.append(batch_loss_state.item())
        running_accuracy_state.append(trainAcc_state)
        running_loss_type.append(batch_loss_type.item())
        running_accuracy_type.append(trainAcc_type)

        del Overall_loss
        del batch_loss_state
        del batch_loss_type
        del label_state
        del label_type
        del predictions_state
        del predictions_type

    
    # 
    model.eval()
    for j, data in enumerate(valid_loader):
            #get batch of data
        inputs, label = data[0].to(device), data[1].to(device)
    
        vlabel_state = torch.zeros(len(label)).to(device)
        vlabel_type = torch.zeros(len(label)).to(device)
        for k in range(len(label)):
            vlabel_state[k] = relabel_state(label[k])
            vlabel_type[k] = relabel_type(label[k]) 
    
        #run model on validation batch
        predictions_state_v, predictions_type_v = model(inputs)
    
            #compute loss
        batch_valid_loss_state = loss_fnc(input=predictions_state_v.squeeze(), target=vlabel_state.long())
        batch_valid_loss_type = loss_fnc(input=predictions_type_v.squeeze(), target=vlabel_type.long())
    
        Overall_loss_v = batch_valid_loss_state + batch_valid_loss_type
            #evaluate
        _, predicted_state_v = torch.max(predictions_state_v.data, 1)
        _,predicted_type_v = torch.max(predictions_type_v.data, 1)
    
        validAcc_state = (vlabel_state == predicted_state_v).sum().item() / 64
        validAcc_type = (vlabel_type == predicted_type_v).sum().item() / 64
        
        if j % eval_every == 0:
            print("epoch: {} |||   vloss type:   {}  validAcc type:   {} |||  vloss state {}  validAcc state:  {}  ".format(e+1, batch_valid_loss_type, validAcc_type, batch_valid_loss_state,validAcc_state))
        
        running_valid_loss.append(Overall_loss_v.item())
        running_valid_accuracy.append((validAcc_state+validAcc_type)/2)
    
        running_valid_loss_state.append(batch_valid_loss_state.item())
        running_valid_accuracy_state.append(validAcc_state)
        running_valid_loss_type.append(batch_valid_loss_type.item())
        running_valid_accuracy_type.append(validAcc_type) 
        del Overall_loss_v
        del batch_valid_loss_state
        del batch_valid_loss_type
        del vlabel_state
        del vlabel_type
        del predicted_state_v
        del predicted_type_v
    #     
    #Overall accuracy
    trainacc_ = sum(running_accuracy) / float(len(running_accuracy))
    train_acc_list.append(trainacc_)
    #state accuracy
    trainacc_state_ = sum(running_accuracy_state) / float(len(running_accuracy_state))
    train_acc_list_state.append(trainacc_state_)
    #type accuracy
    trainAcc_type_ = sum(running_accuracy_type) / float(len(running_accuracy_type))
    train_acc_list_type.append(trainAcc_type_) 
#     
    #             lists for TRAINING losses
    #Overall Loss
    loss_ = sum(running_loss) / float(len(running_loss))
    train_loss_list.append(loss_)
    #State Loss
    loss_state_ = sum(running_loss_state) / float(len(running_loss_state))
    train_loss_list_state.append(loss_state_)
    #Type Loss
    loss_type_ = sum(running_loss_type) / float(len(running_loss_type))
    train_loss_list_type.append(loss_type_) 
        
    #------------------------------------------------------------------------------------
    #             lists for VALIDATION accuracy
    #Overall accuracy
    validacc_ = sum(running_valid_accuracy) / float(len(running_valid_accuracy))
    valid_acc_list.append(validacc_)
    #state accuracy
    validacc_state_ = sum(running_valid_accuracy_state) / float(len(running_valid_accuracy_state))
    valid_acc_list_state.append(validacc_state_)
    #type accuracy
    validAcc_type_ = sum(running_valid_accuracy_type) / float(len(running_valid_accuracy_type))
    valid_acc_list_type.append(validAcc_type_)
    
    #             lists for VALIDATION losses
    #Overall Loss
    valid_loss_ = sum(running_valid_loss) / float(len(running_valid_loss))
    valid_loss_list.append(valid_loss_)
    #State Loss
    valid_loss_state_ = sum(running_valid_loss_state) / float(len(running_valid_loss_state))
    valid_loss_list_state.append(valid_loss_state_)
    #Type Loss
    valid_loss_type_ = sum(running_valid_loss_type) / float(len(running_valid_loss_type))
    valid_loss_list_type.append(valid_loss_type_)
    
    epoch_num.append(e)

In [ ]:
#Plot overall loss vs epoch
plt.figure()
plt.plot(epoch_num, train_loss_list, label='Train')
plt.plot(epoch_num, valid_loss_list, label='Valid')
plt.title('Overall Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\Overall Loss vs. Epoch.png")

#Plot overall Accuracy vs epoch
plt.figure()
plt.plot(epoch_num, train_acc_list, label='Train')
plt.plot(epoch_num, valid_acc_list, label='Valid')
plt.title('Overall Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\Overall Accuracy vs. Epoch.png")

#Plot TYPE loss vs epoch
plt.figure()    
plt.plot(epoch_num, train_loss_list_type, label='Train')
plt.plot(epoch_num, valid_loss_list_type, label='Valid')
plt.title('Type Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\Type Loss vs. Epoch.png")

#Plot TYPE Accuracy vs epoch
plt.figure()
plt.plot(epoch_num, train_acc_list_type, label='Train')
plt.plot(epoch_num, valid_acc_list_type, label='Valid')
plt.title('Type Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\Type Accuracy vs. Epoch.png")

#Plot State loss vs epoch
plt.figure()
plt.plot(epoch_num, train_loss_list_state, label='Train')
plt.plot(epoch_num, valid_loss_list_state, label='Valid')
plt.title('State Loss vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\State Loss vs. Epoch.png")

#Plot TYPE Accuracy vs epoch
plt.figure()
plt.plot(epoch_num, train_acc_list_state, label='Train')
plt.plot(epoch_num, valid_acc_list_state, label='Valid')
plt.title('State Accuracy vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig(r"C:\Users\Mark\Desktop\3rd year 1st term\ECE324\Project\Transfer Learning\Plot\State Accuracy vs. Epoch.png")